In [22]:
import tensorflow as tf

In [141]:
from PIL import Image,ImageOps
import os
import numpy as np
import sys
from PIL import Image

In [44]:
imag1 = Image.open('hate/bringPain.png')

In [45]:
imag1 = imag1.resize((1000,1000))
imag1.save('1.png')

In [46]:
def padding(img, expected_size):
    desired_size = expected_size
    delta_width = desired_size - img.size[0]
    delta_height = desired_size - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)


In [47]:
def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    # print(img.size)
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)

In [51]:
from os import listdir
from os.path import isfile, join
hate_data = [f for f in listdir('hate') if isfile(join('hate', f))]
nothate_data = [f for f in listdir('NotHate') if isfile(join('NotHate', f))]

In [112]:
#add whole path
hate_data_w = np.array(['hate/'+i for i in hate_data])
nothate_data_w = np.array(['NotHate/'+i for i in nothate_data])
np.random.seed(42)
np.random.shuffle(hate_data_w)
np.random.shuffle(nothate_data_w)

In [113]:
train_number_hate = int(len(hate_data_w)*0.8)
train_number_nothate = int(len(nothate_data_w)*0.8)

In [114]:
train_hate = hate_data_w[:train_number_hate]
train_nothate = nothate_data_w[:train_number_nothate]
test_hate = hate_data_w[train_number_hate:]
test_nothate = nothate_data_w[train_number_nothate:]

## Resize images

In [127]:
for i in train_hate:
    imag1 = Image.open(i)
    imag1 = resize_with_padding(imag1, (1000, 1000))
    imag1.save("train/"+i, 'PNG')
for i in train_nothate:
    imag1 = Image.open(i)
    imag1 = resize_with_padding(imag1, (1000, 1000))
    imag1.save("train/"+i, 'PNG')
for i in test_hate:
    imag1 = Image.open(i)
    imag1 = resize_with_padding(imag1, (1000, 1000))
    imag1.save("test/"+i, 'PNG')
for i in test_nothate:
    imag1 = Image.open(i)
    imag1 = resize_with_padding(imag1, (1000, 1000))
    imag1.save("test/"+i, 'PNG')

## Build model

In [152]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow import random
from sklearn.preprocessing import LabelEncoder

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [132]:
seed = 1
np.random.seed(seed)
random.set_seed(seed)
classifier = Sequential()

In [133]:
classifier.add(Conv2D(32, 3, 3, input_shape = (1000, 1000, 3), activation = 'relu'))
classifier.add(MaxPool2D(2,2))
classifier.add(Flatten())
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(1, activation='sigmoid'))

In [134]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [135]:
from keras.preprocessing.image import ImageDataGenerator

In [136]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [138]:
training_set = train_datagen.flow_from_directory('train', target_size=(1000, 1000), batch_size=32, class_mode='binary')

Found 608 images belonging to 3 classes.


In [139]:
test_set = train_datagen.flow_from_directory('test', target_size=(1000, 1000), batch_size=32, class_mode='binary')

Found 160 images belonging to 3 classes.


In [151]:
classifier.fit_generator(training_set, steps_per_epoch=10000, epochs=20, validation_data=test_set, validation_steps=2500, shuffle=False)

/var/folders/0f/brs6kj9d6bbftjh0sk1763yh0000gn/T/ipykernel_24415/3604867387.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(training_set, steps_per_epoch=10000, epochs=20, validation_data=test_set, validation_steps=2500, shuffle=False)


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.